In [ ]:
# Important constants
data_directory = '../tcp_server/out'
model_output_path = '../src/step_counter_model.h'
sample_rate_hz = 100 
window_size_seconds = 0.5
window_size_samples = (int)(sample_rate_hz * window_size_seconds)
step_size_samples = window_size_samples // 2

hidden_layer_size = 12

In [2]:
import pandas as pd
import os

# Total number of data points and steps
length = 0
n_steps = 0

data_frames = []

for file in os.listdir(data_directory):
    if file.startswith('walk'):
        # Read file and add to list
        df = pd.read_csv(os.path.join(data_directory, file))
        data_frames.append(df)

        # Add step count and length to total
        n_steps += df['step'].sum()
        length += len(df)

print("Read all files")
print("Total number of data points:", length)
print("Total number of steps:", n_steps)

Read all files
Total number of data points: 23475
Total number of steps: 362


In [3]:
############################################################################
# Split dataset
############################################################################
import numpy as np

# Input will be a 3x50 array of accelerometer data
# Output will be a 1 if step, 0 if not step
# Data will be split into windows of 50 samples, with a step size of 25 samples

inputData = []
outputData = []

for df in data_frames:
    # Remove final samples which don't fit into window
    # df = df[:len(df)-(len(df) % step_size_samples)]
    
    df = df[0:len(df)-(len(df) % step_size_samples)]

    # Calculate number of windows
    numberOfWindows = int((len(df)-window_size_samples) / step_size_samples) + 1

    for windowNumber in range(numberOfWindows):
        # Get start and end index of window
        startIndex = windowNumber * step_size_samples
        endIndex = startIndex + window_size_samples

        # Input is accelerometer data in window
        inputData.append(df.iloc[startIndex:endIndex, 1:4].values)

        # Output is step count in window
        outputData.append([df.iloc[startIndex:endIndex, 4].sum()])

# Convert to numpy arrays
inputData = np.array(inputData)
outputData = np.array(outputData)

print("Input shape: " + str(inputData.shape))
print("Output shape: " + str(outputData.shape))

Input shape: (931, 50, 3)
Output shape: (931, 1)


In [13]:
# Train model
from tensorflow import keras
from keras import layers

# Initialize model
model = keras.Sequential()
# Convolutional layer with 3 filters, each of size 3, and a stride of 1
model.add(layers.Conv1D(filters = 64, kernel_size=(3), activation='relu', input_shape=(window_size_samples, 3)))
# Max pooling layer with pool size of 2x2 and stride of 2
model.add(layers.MaxPooling1D(pool_size=2, strides=2))
# Convolutional layer with 3 filters, each of size 3, and a stride of 1
model.add(layers.Conv1D(filters = 64, kernel_size=(3), activation='relu', input_shape=(window_size_samples, 3)))
# Max pooling layer with pool size of 2x2 and stride of 2
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# DNN
model.add(layers.Flatten())
model.add(layers.Dense(hidden_layer_size, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
    

# Compile model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

# Train model
model.fit(inputData, outputData, epochs=20, batch_size=1)


Epoch 1/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6116 - loss: 0.3884
Epoch 2/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6051 - loss: 0.3949
Epoch 3/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6277 - loss: 0.3723
Epoch 4/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6076 - loss: 0.3924
Epoch 5/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6213 - loss: 0.3787
Epoch 6/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5681 - loss: 0.4319
Epoch 7/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5920 - loss: 0.4080
Epoch 8/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6243 - loss: 0.3757
Epoch 9/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5802 - loss: 0.4198
Epoch 10/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5924 - loss: 0.4076
Epoch 11/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6199 - loss: 0.3801
Epoch 12/20
931/931 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step